In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Crohns_Disease"
cohort = "GSE123086"

# Input paths
in_trait_dir = "../../input/GEO/Crohns_Disease"
in_cohort_dir = "../../input/GEO/Crohns_Disease/GSE123086"

# Output paths
out_data_file = "../../output/preprocess/Crohns_Disease/GSE123086.csv"
out_gene_data_file = "../../output/preprocess/Crohns_Disease/gene_data/GSE123086.csv"
out_clinical_data_file = "../../output/preprocess/Crohns_Disease/clinical_data/GSE123086.csv"
json_path = "../../output/preprocess/Crohns_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases [study of 13 diseases]"
!Series_summary	"We conducted prospective clinical studies to validate the importance of CD4+ T cells in 13 diseases from the following ICD-10-CM chapters: Neoplasms (breast cancer, chronic lymphocytic leukemia); endocrine, nutritional and metabolic diseases (type I diabetes, obesity); diseases of the circulatory system (atherosclerosis); diseases of the respiratory system (acute tonsillitis, influenza, seasonal allergic rhinitis, asthma); diseases of the digestive system (Crohn’s disease [CD], ulcerative colitis [UC]); and diseases of the skin and subcutaneous tissue (atopic eczema, psoriatic diseases)."
!Series_summary	"Study participants were recruited by clinical specialists based on diagnostic criteria defined by organizations representing each specialist’s discipline. Age and gender matched healthy controls (n = 1

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the Series_overall_design, this dataset contains microarray data from CD4+ T cells
# which would provide gene expression data, not just miRNA or methylation
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait - Crohn's disease appears in row 1 under "primary diagnosis"
trait_row = 1

# For gender - appears in rows 2 and 3, but row 2 seems to be more complete
gender_row = 2 

# For age - appears in rows 3 and 4
age_row = 3

# 2.2 Data Type Conversion
def convert_trait(value):
    if not isinstance(value, str):
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Matching trait (Crohn's Disease)
    if "CROHN" in value.upper():
        return 1
    # Healthy controls should be 0
    elif "HEALTHY" in value.upper() or "CONTROL" in value.upper():
        return 0
    # Other diseases are not relevant for our binary classification
    else:
        return None

def convert_gender(value):
    if not isinstance(value, str):
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Female is 0, Male is 1
    if value.upper() == "FEMALE":
        return 0
    elif value.upper() == "MALE":
        return 1
    # If it's a diagnosis2 field, return None as it's not gender data
    elif "DIAGNOSIS2" in value.upper():
        return None
    else:
        return None

def convert_age(value):
    if not isinstance(value, str):
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float for continuous age
    try:
        return float(value)
    except:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering using validate_and_save_cohort_info
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM3494884': [nan, 56.0, 1.0], 'GSM3494885': [nan, nan, nan], 'GSM3494886': [nan, 20.0, 0.0], 'GSM3494887': [nan, 51.0, 0.0], 'GSM3494888': [nan, 37.0, 1.0], 'GSM3494889': [nan, 61.0, 1.0], 'GSM3494890': [nan, nan, nan], 'GSM3494891': [nan, 31.0, 1.0], 'GSM3494892': [nan, 56.0, 0.0], 'GSM3494893': [nan, 41.0, 0.0], 'GSM3494894': [nan, 61.0, 0.0], 'GSM3494895': [nan, nan, nan], 'GSM3494896': [nan, 80.0, 1.0], 'GSM3494897': [nan, 53.0, 1.0], 'GSM3494898': [nan, 61.0, 1.0], 'GSM3494899': [nan, 73.0, 1.0], 'GSM3494900': [nan, 60.0, 1.0], 'GSM3494901': [nan, 76.0, 1.0], 'GSM3494902': [nan, 77.0, 0.0], 'GSM3494903': [nan, 74.0, 0.0], 'GSM3494904': [nan, 69.0, 1.0], 'GSM3494905': [nan, 77.0, 0.0], 'GSM3494906': [nan, 81.0, 0.0], 'GSM3494907': [nan, 70.0, 0.0], 'GSM3494908': [nan, 82.0, 0.0], 'GSM3494909': [nan, 69.0, 0.0], 'GSM3494910': [nan, 82.0, 0.0], 'GSM3494911': [nan, 67.0, 0.0], 'GSM3494912': [nan, 67.0, 0.0], 'GSM3494913': [nan, 78.0, 0.0], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')

Gene data dimensions: 22881 genes × 166 samples


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# The identifiers appear to be numerical values (1, 2, 3, etc.)
# These are not standard human gene symbols, which are typically alphanumeric 
# (like BRCA1, TP53, etc.)
# These appear to be probe IDs or some other form of identifiers that would
# need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# 3. Preview the gene annotation dataframe
print("Gene annotation dataframe column names:")
print(gene_annotation.columns)

# Preview the first few rows to understand the data structure
print("\nPreview of gene annotation data:")
annotation_preview = preview_df(gene_annotation, n=3)
print(annotation_preview)

# Maintain gene availability status as True based on previous steps
is_gene_available = True


Gene annotation dataframe column names:
Index(['ID', 'ENTREZ_GENE_ID', 'SPOT_ID'], dtype='object')

Preview of gene annotation data:
{'ID': ['1', '2', '3'], 'ENTREZ_GENE_ID': ['1', '2', '3'], 'SPOT_ID': [1.0, 2.0, 3.0]}


### Step 6: Gene Identifier Mapping

In [7]:
# Let's examine the gene_annotation data more carefully to understand the structure
print("Gene annotation first few rows:")
print(gene_annotation.head())

# Check what's in the ENTREZ_GENE_ID column - we need actual gene identifiers
print("\nSample values in ENTREZ_GENE_ID column:")
print(gene_annotation['ENTREZ_GENE_ID'].head(10))

# The issue is that we need proper gene symbols, not just Entrez IDs
# Let's check if we have access to proper gene symbols by fetching the platform annotation
# from the SOFT file

# Parse the SOFT file to get platform information including gene symbols
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for sections containing gene symbol information
print("\nCheck if gene symbols are available in the SOFT file:")
gene_symbol_lines = [line for line in soft_content.split('\n') if 'gene_symbol' in line.lower()][:5]
print(gene_symbol_lines)

# If we don't find gene symbols directly, we'll use the Entrez Gene IDs as identifiers
# since they can be mapped to gene symbols later

# Create a mapping dataframe with ID and Entrez Gene ID
gene_mapping = gene_annotation[['ID', 'ENTREZ_GENE_ID']].copy()
gene_mapping.rename(columns={'ENTREZ_GENE_ID': 'Gene'}, inplace=True)

# Filter out any rows with missing values
gene_mapping = gene_mapping.dropna()

# Print a preview of the cleaned mapping
print("\nCleaned gene mapping:")
print(gene_mapping.head())
print(f"Mapping shape after cleaning: {gene_mapping.shape}")

# Apply the gene mapping to convert probe-level measurements to gene expression data
# We'll use the standard function but make sure our Gene column has proper values
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print statistics about the resulting gene data
print(f"\nGene expression data after mapping:")
print(f"Number of genes: {gene_data.shape[0]}")
print(f"Number of samples: {gene_data.shape[1]}")

# Check the first few gene identifiers - they should be Entrez IDs
if gene_data.shape[0] > 0:
    print("First 5 gene identifiers:")
    print(gene_data.index[:5])
else:
    print("No genes were mapped successfully.")

# For this dataset, since we don't have access to proper gene symbols, 
# we'll treat the Entrez Gene IDs as our gene identifiers
# Later normalization can map these to standard gene symbols if needed

# Verify we have non-empty gene expression data
is_gene_available = gene_data.shape[0] > 0


Gene annotation first few rows:
   ID ENTREZ_GENE_ID  SPOT_ID
0   1              1      1.0
1   2              2      2.0
2   3              3      3.0
3   9              9      9.0
4  10             10     10.0

Sample values in ENTREZ_GENE_ID column:
0     1
1     2
2     3
3     9
4    10
5    12
6    13
7    14
8    15
9    16
Name: ENTREZ_GENE_ID, dtype: object



Check if gene symbols are available in the SOFT file:


[]



Cleaned gene mapping:
   ID Gene
0   1    1
1   2    2
2   3    3
3   9    9
4  10   10
Mapping shape after cleaning: (3822578, 2)



Gene expression data after mapping:
Number of genes: 0
Number of samples: 166
No genes were mapped successfully.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check if gene data is available after mapping
if gene_data.shape[0] == 0:
    print("Error: Gene expression matrix is empty after mapping.")
    # Mark the dataset as not usable due to lack of gene expression data
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # No usable gene data
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note="Failed to map probe IDs to gene symbols. The annotation format may not be compatible with the extraction methods."
    )
    print("Dataset deemed not usable due to lack of gene expression data.")
else:
    # Only proceed with normalization if we have gene data
    print("Normalizing gene symbols...")
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_normalized.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
    
    # Extract clinical features from the original data source
    print("Extracting clinical features from the original source...")
    # Get background information and clinical data again
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Extracted clinical features preview:")
    print(preview_df(selected_clinical_df))
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if the linked data has adequate data
    if linked_data.shape[0] == 0 or linked_data.shape[1] <= 4:  # 4 is an arbitrary small number
        print("Error: Linked data has insufficient samples or features.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to properly link gene expression data with clinical features."
        )
        print("Dataset deemed not usable due to linking failure.")
    else:
        # Handle missing values systematically
        print("Handling missing values...")
        linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data_clean.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data_clean.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
            print("Dataset deemed not usable as all samples were filtered out.")
        else:
            # Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data_final = judge_and_remove_biased_features(linked_data_clean, trait)
            
            # Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data_final,
                note="Dataset contains gene expression data for Crohn's Disease patients, examining response to Infliximab treatment."
            )
            
            # Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_final.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
                print(f"Final dataset shape: {linked_data_final.shape}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")

Error: Gene expression matrix is empty after mapping.
Abnormality detected in the cohort: GSE123086. Preprocessing failed.
A new JSON file was created at: ../../output/preprocess/Crohns_Disease/cohort_info.json
Dataset deemed not usable due to lack of gene expression data.
